In [1]:
import requests
import time
import pandas as pd
from janome.tokenizer import Tokenizer
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
URL = "https://www.aozora.gr.jp/cards/000879/files/128_15261.html"

In [3]:
book_source = requests.get(URL)
soup = BeautifulSoup(book_source.content, "html.parser")
book_text = soup.find("div",{"class":"main_text"}).get_text()
book_title = soup.find("h1").get_text()
book_author = soup.find("h2").get_text()
file_name = f'{book_author} - {book_title}.csv'
print(f'Book loaded: {book_title} by {book_author}')

Book loaded: 羅生門 by 芥川龍之介


In [4]:
def extract_wordlist(text):
    t = Tokenizer()
    wordlist = []

    for token in t.tokenize(text):
        if "名詞" in token.part_of_speech or (token.part_of_speech.startswith("動詞") and not token.part_of_speech.startswith("助動詞")) or "形容詞" in token.part_of_speech or "代名詞" in token.part_of_speech:
            wordlist.append(token.base_form)
    return wordlist

In [5]:
wordlist = extract_wordlist(book_text)
wordlist = Counter(wordlist)
wordlist = pd.DataFrame(wordlist.items(), columns = ['Word', 'Count']).sort_values(by = 'Count', ascending = False)
print(wordlist)

    Word  Count
31    する     78
11    ゐる     59
5     下人     44
2      事     31
42     云     30
..   ...    ...
301    惧      1
300   人目      1
299    患      1
298   雨風      1
688   九月      1

[689 rows x 2 columns]


In [18]:
requests.get("https://jlpt-vocab-api.vercel.app/api/words?word=人目").json()

{'total': 1,
 'offset': 0,
 'limit': 10,
 'words': [{'word': '人目',
   'meaning': 'glimpse; public gaze',
   'furigana': 'じんもく',
   'romaji': 'jinmoku',
   'level': 1}]}

In [19]:
requests.get("https://jlpt-vocab-api.vercel.app/api/words?word=人目").json()['words'][0]['level']

1

In [28]:
def make_api_request(word):
    url = f"https://jlpt-vocab-api.vercel.app/api/words?word={word}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except (requests.exceptions.RequestException, requests.exceptions.Timeout):
        print(f"Timeout for word: {word}. Retrying in 5 seconds...")
        time.sleep(5)
        return make_api_request(word)

In [29]:
def get_word_info(wordlist):
    readings = []
    levels = []
    t = Tokenizer()

    for word in wordlist:
        response = make_api_request(word)
        tokens = list(t.tokenize(word))

        reading = tokens[0].reading
        
        try:
            level = response['words'][0]['level']
        except (IndexError, KeyError):
            level = 'unknown'

        readings.append(reading)
        levels.append(level)

    return readings, levels

In [30]:
testlist = ['聞く', '焼く', '本気']
readings, levels = get_word_info(testlist)
print(readings)
print(levels)

['キク', 'ヤク', 'ホンキ']
[5, 4, 1]


In [ ]:
wordlist['Reading'], wordlist['Level'] = get_word_info(wordlist['Word'])

In [ ]:
wordlist.to_csv(file_name, index = False)